In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import * 
from pyspark.sql import SparkSession, DataFrame as SparkDataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import col,isnan, when, count
from pyspark import SparkContext
from pyspark.sql import SQLContext
import json
from functools import reduce
import sys
from cassandra.cluster import Cluster
import os
# from mock.tasks import adiciona_carro
cluster = Cluster(['cassandra'])
session = cluster.connect()

ss = SparkSession.builder.appName("test").getOrCreate()
sql = SQLContext(ss)

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
session.execute("USE simulacao")

In [3]:
session.execute("select count(*) from simulacao")[0]

/tmp/ipykernel_8044/795349805.py:1: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  session.execute("select count(*) from simulacao")[0]


Row(count=105)

In [4]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = 142 ALLOW FILTERING;")

In [5]:
result[0]

/tmp/ipykernel_8044/223460175.py:1: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  result[0]


IndexError: list index out of range

In [ ]:
r = list(result)

In [ ]:
r

In [ ]:
df = ss.createDataFrame(r)

df.show()

In [ ]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = 143 ALLOW FILTERING;")
r = list(result)
df2 = ss.createDataFrame(r)
df2.show()

In [ ]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = 144 ALLOW FILTERING;")
r = list(result)
df25 = ss.createDataFrame(r)
df25.show()

In [ ]:
df_empty = ss.createDataFrame([], "placa: string, posicao: int, rodovia: string, tempo_da_simulacao: int, velocidade: double, aceleracao: double, posicao_prevista: double, acima_vel: boolean")

In [ ]:
import json
params = json.load(open('./mock/parametros.json'))

In [ ]:
p = [[key]+list(params[key].values()) for key in params.keys()]

In [ ]:
p = ss.createDataFrame(p, ["Rodovia"]+list(params[list(params.keys())[0]].keys()))

p.toPandas()

In [ ]:
fps = 30

def processa_carro(DadosNovos, DadosCarros, colision_tolerance, colision_tolerance_quad, Parametros):
    DadosCarros = DadosCarros.drop('aplicaMulta')
    DadosNovos = DadosNovos.select(F.col('pos_y').alias('posicao_nova'), F.col('rodovia'),
                      F.col('tempo_da_simulacao').alias('tempo_da_simulacao_novo'),F.col('placa'))
    data_joined = DadosCarros.join(DadosNovos, on="placa", how='right')
    data_joined = data_joined.withColumn("diferenca_de_posicao", col("posicao_nova") - col("posicao"))
    data_joined = data_joined.withColumn("diferenca_de_horario", (col("tempo_da_simulacao_novo") - col("tempo_da_simulacao"))*fps)
    
    data_joined = data_joined.withColumnRenamed("velocidade", "velocidade_antiga")
    data_joined = data_joined.withColumn("velocidade", col("diferenca_de_posicao") / col("diferenca_de_horario"))
    data_joined = data_joined.withColumn("diferenca_de_velocidade", col("velocidade") - col("velocidade_antiga"))
    data_joined = data_joined.withColumn("aceleracao", col("diferenca_de_velocidade") / col("diferenca_de_horario"))
    
    data_joined = data_joined.drop("velocidade_antiga", "posicao", "tempo_da_simulacao",
                     "diferenca_de_posicao", "diferenca_de_horario",
                     "diferenca_de_velocidade")
    data_joined = data_joined.withColumnRenamed("posicao_nova", "posicao")
    data_joined = data_joined.withColumnRenamed("tempo_da_simulacao_novo", "tempo_da_simulacao")
    data_joined = data_joined.withColumn("posicao_prevista", col("posicao")\
                           + col("velocidade")*colision_tolerance\
                           + col("aceleracao")*colision_tolerance_quad)

    Velocidades_Maximas = Parametros.select(F.col('rodovia'), F.col("VelocidadeMaxima"))
    data_joined = data_joined.join(Velocidades_Maximas, on="rodovia", how="left")

    acima_vel_df = data_joined.select(F.col('placa'), F.col('acima_vel').alias('acima_vel_antigo'))
    data_joined = data_joined.withColumn("acima_vel", F.when(F.abs(data_joined["velocidade"]) > F.abs(fps/data_joined["VelocidadeMaxima"]), 1).otherwise(0))
    acima_vel_df = acima_vel_df.join(data_joined.select(F.col('placa'), F.col('acima_vel').alias('acima_vel_novo')), on='placa', how="left")
    acima_vel_df = acima_vel_df.withColumn("aplicaMulta", (F.col('acima_vel_antigo')==0) &  (F.col('acima_vel_novo')==1) )

    data_joined = data_joined.join(acima_vel_df.select(F.col('placa'), F.col('aplicaMulta')), on='placa', how='left')
    data_joined = data_joined.drop("VelocidadeMaxima")
    return data_joined
    

In [ ]:
df3 = processa_carro(df, df_empty, 1, 0.5, p)

In [ ]:
df3 = processa_carro(df2, df3, 1, 0.5, p)

In [ ]:
df4 = processa_carro(df25, df3, 1, 0.5, p).toPandas()

In [ ]:
df4

In [ ]:
def cria_colunas_multas(df):
    for i in range(10):
        df = df.withColumn("multa_"+str(i), F.lit(0))
    return df

In [ ]:
cria_colunas_multas(df4).toPandas()